In [70]:
import os
import sys
import subprocess
from contextlib import contextmanager
import signal
import filecmp
import difflib
import shutil

# Configurations
toolchain_path = "/home/r1c0/riscv/bin/riscv64-unknown-linux-gnu-"
compiler_path = "./build/SYSY-compiler"
test_folder="./testcases"
sylib_path = "./RISCVLib/sylib.c"
# xxx.sy 会被复制到./function_test/test.c
temp_path = "./function_test"
pass_args=[]
Tolerance = True

CE_list = [] # Compile Error
AE_list = [] # Assembler Error
LE_list = [] # Linker Error
Time_Out = []
WA_list = []
AC_list = []
TLE_list = []
BadTest_list = []

Prob_list = []
test_list = []

In [71]:
# grab all the testcases
# find files recursively
def GrabFile():
    for root, dirs, files in os.walk(test_folder):
        for file in files:
            if file.endswith(".sy"):
                test_list.append(os.path.join(root, file))

In [72]:
def CopyFileToTemp(test):
    # No Bad Test Now
    shutil.copy(test,temp_path+"/test.c")
    if os.path.exists(test[:-2]+"in"):
        shutil.copy(test[:-2]+"in",temp_path+"/test.in")
    if os.path.exists(test[:-2]+"out"):
        shutil.copy(test[:-2]+"out",temp_path+"/test.out")

In [73]:
def TryCompile(original_case_path):
    compile_args=[compiler_path, temp_path + "/test.c"]
    for arg in pass_args:
        compile_args.append(arg)
    try:
        ret = subprocess.run(compile_args,timeout=10)
    except subprocess.TimeoutExpired:
        Time_Out.append(original_case_path)
        print("Timeout: "+original_case_path)
        return False
    if ret.returncode != 0:
        CE_list.append(original_case_path)
        print("CE: "+original_case_path)
        return False
    return True

In [74]:
def TryAssemble(original_case_path):
    try:
        ret = subprocess.run([toolchain_path+"as", temp_path + "/test.s", "-o", temp_path + "/test.o"],timeout=10)
    except subprocess.TimeoutExpired:
        print("IMPOSSIBLE, AS TIMEOUT")
        exit(-1)
    if ret.returncode != 0:
        AE_list.append(original_case_path)
        print("AE: "+original_case_path)
        return False
    return True

In [75]:
def TryLink(orginal_case_path):
    try:
        ret = subprocess.run([toolchain_path+"gcc", temp_path + "/test.o", temp_path+"/sylib.o", "-o", temp_path + "/test"],timeout=10)
    except subprocess.TimeoutExpired:
        print("IMPOSSIBLE, LD TIMEOUT")
        exit(-1)
    if ret.returncode != 0:
        LE_list.append(orginal_case_path)
        print("LE: "+orginal_case_path)
        return False
    return True

In [76]:

def RunTest(original_case_path):
    try:
        if not os.path.exists(temp_path+"/test.in"):
            ret = subprocess.run(["qemu-riscv64", temp_path + "/test"],stdout=subprocess.PIPE,stderr=subprocess.PIPE,timeout=10)
        else:
            ret = subprocess.run(["qemu-riscv64", temp_path + "/test"],stdin=open(temp_path+"/test.in"),stdout=subprocess.PIPE,stderr=subprocess.PIPE,timeout=60)
    except subprocess.TimeoutExpired:
        print("TIMEOUT ERROR: "+original_case_path)
        TLE_list.append(original_case_path)
        return False
    
    if not os.path.exists(temp_path + "/test.out"):
        BadTest_list.append(original_case_path)
        return False
    else:
        out_file=temp_path + "/test.out"
    
    dump_str=ret.stdout.decode()
    # dump_str1 = ret1.stdout.decode()
    # remove whitesspace in the end
    # dump_str=dump_str.rstrip()
    # if dump_str1 and not dump_str1.endswith('\n'):
    #     dump_str1 += "\n"
    # if not dump_str1.endswith(''):
    #     print("RISCV Test Error")
    if dump_str and not dump_str.endswith('\n'):
        dump_str += "\n"
    dump_str += str(ret.returncode) + "\n"
    std_output=open(out_file).read()
    diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
    if(len(list(diff))!=0):
        print("Wrong Answer: "+original_case_path)
        WA_list.append(original_case_path)
        print(original_case_path)
        return False
    
    return True

In [77]:
def Tolerant(FuncList,original_file_name):
    for func in FuncList:
        ret = func(original_file_name)
        if ret == False and Tolerance == False:
            print("No Tolerence, exit")
            sys.exit(-1)
        if ret == False:
            return False
    return True

In [78]:
def TestSingleCase(test):
    CopyFileToTemp(test)
    if Tolerant([TryCompile,TryAssemble,TryLink, RunTest],test)==False:
        return False
    AC_list.append(test)
    return True

In [79]:
# Compile the sylib.c
subprocess.run([toolchain_path+"gcc", sylib_path, "-c", "-o", temp_path+"/sylib.o"])

CompletedProcess(args=['/home/r1c0/riscv/bin/riscv64-unknown-linux-gnu-gcc', './RISCVLib/sylib.c', '-c', '-o', './function_test/sylib.o'], returncode=0)

In [80]:
if len(Prob_list) == 0:
    GrabFile()
else:
    test_list=Prob_list
    Prob_list.clear()
    
for test in test_list:
    if test.endswith(".sy"):
        if TestSingleCase(test):
            print("Pass: "+test)

Pass: ./testcases/functional/50_short_circuit.sy
Wrong Answer: ./testcases/functional/69_expr_eval.sy
./testcases/functional/69_expr_eval.sy
Pass: ./testcases/functional/49_if_complex_expr.sy
Pass: ./testcases/functional/11_add2.sy
Wrong Answer: ./testcases/functional/64_calculator.sy
./testcases/functional/64_calculator.sy
Wrong Answer: ./testcases/functional/95_float.sy
./testcases/functional/95_float.sy
Wrong Answer: ./testcases/functional/61_sort_test7.sy
./testcases/functional/61_sort_test7.sy
Pass: ./testcases/functional/29_break.sy
Pass: ./testcases/functional/45_comment1.sy
Pass: ./testcases/functional/47_hex_oct_add.sy
Wrong Answer: ./testcases/functional/55_sort_test1.sy
./testcases/functional/55_sort_test1.sy
Wrong Answer: ./testcases/functional/05_arr_defn4.sy
./testcases/functional/05_arr_defn4.sy
Wrong Answer: ./testcases/functional/58_sort_test4.sy
./testcases/functional/58_sort_test4.sy
Wrong Answer: ./testcases/functional/08_const_array_defn.sy
./testcases/functional/0

./function_test/test.s: Assembler messages:
./function_test/test.s:76: Error: illegal operands `sd ft0,-336(s0)'
./function_test/test.s:78: Error: illegal operands `sd ft0,-464(s0)'
./function_test/test.s:81: Error: illegal operands `sd ft0,-184(s0)'
./function_test/test.s:83: Error: illegal operands `sd ft0,-192(s0)'
./function_test/test.s:87: Error: illegal operands `sd ft0,-200(s0)'
./function_test/test.s:91: Error: illegal operands `sd ft0,-344(s0)'
./function_test/test.s:96: Error: illegal operands `sd ft0,-352(s0)'
./function_test/test.s:98: Error: illegal operands `sd ft0,-360(s0)'
./function_test/test.s:100: Error: illegal operands `sd ft0,-368(s0)'
./function_test/test.s:103: Error: illegal operands `sd ft0,-376(s0)'
./function_test/test.s:107: Error: illegal operands `sd ft0,-384(s0)'
./function_test/test.s:112: Error: illegal operands `sd ft0,-392(s0)'
./function_test/test.s:117: Error: illegal operands `sd ft0,-208(s0)'
./function_test/test.s:124: Error: illegal operands `s

AE: ./testcases/hidden_functional/39_fp_params.sy
Wrong Answer: ./testcases/hidden_functional/13_LCA.sy
./testcases/hidden_functional/13_LCA.sy
Pass: ./testcases/hidden_functional/33_multi_branch.sy
Wrong Answer: ./testcases/hidden_functional/30_many_dimensions.sy
./testcases/hidden_functional/30_many_dimensions.sy
Wrong Answer: ./testcases/hidden_functional/29_long_line.sy
./testcases/hidden_functional/29_long_line.sy
Wrong Answer: ./testcases/hidden_functional/18_prim.sy
./testcases/hidden_functional/18_prim.sy
Timeout: ./testcases/hidden_functional/32_many_params3.sy
Wrong Answer: ./testcases/hidden_functional/22_matrix_multiply.sy
./testcases/hidden_functional/22_matrix_multiply.sy
Pass: ./testcases/hidden_functional/04_break_continue.sy
Wrong Answer: ./testcases/hidden_functional/35_math.sy
./testcases/hidden_functional/35_math.sy
Wrong Answer: ./testcases/hidden_functional/23_json.sy
./testcases/hidden_functional/23_json.sy
Wrong Answer: ./testcases/hidden_functional/37_dct.sy
./

./function_test/test.s: Assembler messages:
./function_test/test.s:60: Error: illegal operands `fmv.s fa0,undef'


Wrong Answer: ./testcases/final_performance/04_spmv1.sy
./testcases/final_performance/04_spmv1.sy
Wrong Answer: ./testcases/final_performance/01_mm1.sy
./testcases/final_performance/01_mm1.sy
Wrong Answer: ./testcases/final_performance/gameoflife-oscillator.sy
./testcases/final_performance/gameoflife-oscillator.sy
Wrong Answer: ./testcases/final_performance/fft0.sy
./testcases/final_performance/fft0.sy
Wrong Answer: ./testcases/final_performance/shuffle0.sy
./testcases/final_performance/shuffle0.sy
AE: ./testcases/final_performance/recursive_call_2.sy


./function_test/test.s: Assembler messages:
./function_test/test.s:60: Error: illegal operands `fmv.s fa0,undef'


Wrong Answer: ./testcases/final_performance/fft2.sy
./testcases/final_performance/fft2.sy
Wrong Answer: ./testcases/final_performance/shuffle1.sy
./testcases/final_performance/shuffle1.sy
Wrong Answer: ./testcases/final_performance/transpose2.sy
./testcases/final_performance/transpose2.sy
AE: ./testcases/final_performance/recursive_call_1.sy


./function_test/test.s: Assembler messages:
./function_test/test.s:60: Error: illegal operands `fmv.s fa0,undef'


Wrong Answer: ./testcases/final_performance/matmul1.sy
./testcases/final_performance/matmul1.sy
Wrong Answer: ./testcases/final_performance/sl1.sy
./testcases/final_performance/sl1.sy
Wrong Answer: ./testcases/final_performance/large_loop_array_3.sy
./testcases/final_performance/large_loop_array_3.sy
Wrong Answer: ./testcases/final_performance/04_spmv2.sy
./testcases/final_performance/04_spmv2.sy


In [81]:
# print all wrong tests and mv one of them to temp_folder for debugging
print(len(AC_list))
print(len(WA_list))
print(len(CE_list))
print(len(AE_list))
print(len(LE_list))
print(len(Time_Out))
print(len(TLE_list))
print(len(BadTest_list))

68
150
0
4
0
11
2
0


In [87]:
def MV2WorkSpace(CasesList,k):
    case = CasesList[k]
    CopyFileToTemp(case)
    print(case)
    return
        
MV2WorkSpace(WA_list,4)

./testcases/functional/55_sort_test1.sy
